In [2]:
pip install opencv-python opencv-contrib-python

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 1.0 MB/s eta 0:00:0000:0100:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 865.6 kB/s eta 0:00:0000:0100:02
Note: you may need to restart the kernel to use updated packages.


In [1]:
from xml.dom.minidom import parse
import os
import cv2
import torch

def generate_batch_size(data, batch_size):
    images, gts = data['images'], data['gts']
    images = torch.stack(images, dim=0)
    len_images = len(images)
    num_batch = len_images // batch_size
    batch_images = []
    batch_gits = []
    for i in range(num_batch):
        batch_data = images[i * batch_size: (i + 1) * batch_size]
        batch_images.append(batch_data)
        boxes = []
        for j in range(i * batch_size, (i + 1) * batch_size):
            box = gts[j][:, 1:]
            label = gts[j][:, 0]
            # ------- 将每一张图片的所有真实框的位置信息与它的标签一起放置
            boxes.append({"boxes": box, "labels": label.long()})
        batch_gits.append(boxes)
    return batch_images, batch_gits

In [ ]:
import torch
import torchvision
from torchvision.models import MobileNet_V2_Weights
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import pynvml


device = torch.device("cuda:0")

backbone = torchvision.models.mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT).features
backbone = backbone.to(device)
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((64, 128, 256),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)
roi_pooler = roi_pooler.to(device)

model = FasterRCNN(backbone,
                   num_classes=21,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)
model = model.to(device)

data = torch.load("./data/dog_train.pth", weights_only=False)
batch_images, batch_gits = generate_batch_size(data, 6)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 显存监控（假如20batch_size 显存未满，则向上调解)）
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
# ------------------------训练------------------------
model.train()
epochs = 10000
for epoch in range(epochs):
    losses = []
    for images, gts in zip(batch_images, batch_gits):
        images = images.to(device)
        optimizer.zero_grad()
        result = model(images.float(), gts)
        loss_classifier = result['loss_classifier']
        loss_box_reg = result['loss_box_reg']
        loss_objectness = result['loss_objectness']
        loss_rpn_box_reg = result['loss_rpn_box_reg']
        loss = loss_classifier + loss_box_reg + loss_objectness + loss_rpn_box_reg
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    total_loss = sum(losses) / len(losses)
    print(f"epoch {epoch + 1} / {epochs} -- loss:{total_loss:.4f}")

    meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print(f'GPU in use {(meminfo.used / 1024 ** 3):.4f}/{meminfo.total / 1024 ** 3} G')


epoch 1 / 10000 -- loss:1.0459
GPU in use 11.1883/12.0 G
epoch 2 / 10000 -- loss:0.6605
GPU in use 11.1883/12.0 G
epoch 3 / 10000 -- loss:0.5555
GPU in use 11.1883/12.0 G
epoch 4 / 10000 -- loss:0.5055
GPU in use 11.1883/12.0 G
epoch 5 / 10000 -- loss:0.4788
GPU in use 11.1883/12.0 G
epoch 6 / 10000 -- loss:0.4466
GPU in use 11.1883/12.0 G
epoch 7 / 10000 -- loss:0.4207
GPU in use 11.1883/12.0 G
epoch 8 / 10000 -- loss:0.4138
GPU in use 11.1883/12.0 G
epoch 9 / 10000 -- loss:0.3965
GPU in use 11.1883/12.0 G
epoch 10 / 10000 -- loss:0.3756
GPU in use 11.1883/12.0 G
epoch 11 / 10000 -- loss:0.3472
GPU in use 11.1883/12.0 G
epoch 12 / 10000 -- loss:0.3277
GPU in use 11.1883/12.0 G
